# MLflow Local Run Edition

*It is recommended you start a virtual environment before running the session*

For MacOS:

After you are in the right directory,

`python3 -m venv <environment name>` <br>
`source <environment name>/bin/activate`

**Run `pip install mlflow` or `conda install mlflow` if you are using conda to install mlflow**

In [32]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

**In your terminal, set up mlflow server by typing `mlflow server`, by default, http://127.0.0.1:5000 will be the tracking uri**

**You can go to the link http://127.0.0.1:5000 to gain access to the mlflow ui**

In [33]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [34]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

In [35]:
mlflow.set_experiment("Example Experiment")

<Experiment: artifact_location='mlflow-artifacts:/928213069348323692', creation_time=1704424215952, experiment_id='928213069348323692', last_update_time=1704424215952, lifecycle_stage='active', name='Example Experiment', tags={}>

**You should see a new experiment show in the mlflow ui, on your local device, you should also see a "mlruns" directory**

In [ ]:
import os
# Start an MLflow run
mlflow.autolog()
with mlflow.start_run(run_name='run1'):
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))
    
    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path='iris_model',
        signature=signature,
        input_example=X_train,
        registered_model_name="iris_model", # register the model directly with log_model function
    )

**Navigate to 'Model' tab in the mlflow ui (top bar), you should see a newly registered model named "iris_model"**

**Now go to the terminal and input the following command**

`export MLFLOW_TRACKING_URI="http://127.0.0.1:5000"` This line set up connection with mlflow

`$env:MLFLOW_TRACKING_URI="http://127.0.0.1:5000"` (if you are using windows)


`mlflow models serve -m "models:/iris_model/1" --port 5002` This line serve the model on port 5002 (it is required to specify a port)

*if you ran into an error saying pyenv is not found, follow the download instructions to download pyenv*

<br>

**The model is ready to be used if you see something similar to this `[2024-01-05 11:49:40 +0800] [85139] [INFO] Listening at: http://127.0.0.1:5002 (85139)`**

**To use the model to make predictions, you can use the `curl` command**

`curl http://127.0.0.1:5002/invocations -H 'Content-Type: application/json' -d '{
  "dataframe_split": {
      "data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]
  }
}'`

More about the curl command: https://mlflow.org/docs/latest/models.html#local-model-deployment

**Alternatively, once you set up the serving endpoint, instead of using curl command, you can also do a post request as follow**

In [45]:
import requests

endpoint = "http://localhost:5002/invocations" # make sure the port number matches the one you input in terminal
data = {
    "dataframe_split": {  
        "data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]   
    }
}
# do a post request
response = requests.post(endpoint, json=data)
print(response.json())

{'predictions': [1, 0]}


#### Docker 

(noted that you will need docker installed on your computer to use this feature https://www.docker.com/products/docker-desktop/)


**Instead of using serve to run the model, you can build a docker image instead** 

`mlflow models build-docker --model-uri "models:/iris_model/1" --name "iris"` This line build the docker image

`docker run -p 5002:8080 iris` Run the docker container on the respective port

**To use the model, you can use the exact same command as you do in model serving**

`curl http://127.0.0.1:8080/invocations -H 'Content-Type: application/json' -d '{"dataframe_split": {"data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]}}'`

**!Note! Since windows does not support single quote, you have to use this command instead if you are using windows**

`curl http://127.0.0.1:8080/invocations -H "Content-Type: application/json" -d "{\"dataframe_split\": {\"data\": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]}}"`